# Exact Diagonalization Prerequisites
<b> Christina Lee

Category: Grad</b>

I was going to go at exact diagonalization of a 1D Heisenburg Spin Chain all in one post, but then I realized, even for a grad level post, I need to explain a lot of stuff.  So here I'm going to examine some of the tricks we will use when actually performing exact diagonalization.  

So what is a 1D Heisenburg Spin $\frac{1}{2}$ Chain? 

### Spin $\frac{1}{2} $
At each site, we can have a particle pointing up $| \uparrow \rangle$, down $|\downarrow \rangle$, or some super-position of the two.  

### Heisenburg
Our spin has three degrees of freedom.  Our Hamiltonian has the general form
\begin{equation}
{\cal H} = \sum_{\langle i,j \rangle} J_x S_i^x S_j^x + J_y S_i^y S_j^y + J_z S_i^z S_j^z
\end{equation}

In the limit of $J_z \rightarrow 0$, we regain the XY model, and when only one coupling constant remains we regain the Ising model.  

### 1D Chain
Spin only couples to two neighbors.

### One easy initialization cell.

`n`: The number of spins we will deal with.  
$2^n$: Dimension of our Hilbert Space.  Or more simply, the number of different eigenstates our system will have.

In [7]:
n=3
nstates=2^n

8

So Exact Diagonalization is often memory limited.  Thus we want to represent our states in the most compact format possible.  Luckily, if we are dealing with spin $\frac{1}{2}$, we can just use the `0`'s ($|\downarrow \rangle$) and `1`'s ($|\uparrow \rangle$) of the machine.  If you are dealing with higher spin, you can use base 3, 4, etc...  Part of the reason I needed to create this seperate post was to examine working with binary data.

We will keep our states stored as Int, but Julia has operations we can perform to look at the binary format and change the bits.

In [8]:
# psi is an arrow of all our wavefunctions
psi=collect(0:(nstates-1))

# Lets look at each state both in binary and base 10
println("binary form \t integer")
for p in psi
    println(bin(p,n+1),"\t\t ",p)
end

binary form 	 interger
0000		 0
0001		 1
0010		 2
0011		 3
0100		 4
0101		 5
0110		 6
0111		 7


For handy reference, since we will be using them a lot in our calculations, lets store all the powers of two in an array. They are our placeholders.

In [13]:
powers2=2.^collect(0:n)

println("binary form \t integer")
for p in powers2
    println(bin(p,n+1),"\t\t ",p)
end

binary form 	 integer
0001		 1
0010		 2
0100		 4
1000		 8


### & And Operation and Computing Magnetization

We could continue to look at the binary format of a number by calling `bin`, but that converts the number to an array of strings.  So instead we want to perform bitwise operations to determine what the binary format looks like in terms of numbers.

Julia supports bitwise <b>not, and, xor </b> (exclusive or), logical shift right, arithmetic shift right, and logical/ arithmetic shift left.  For our purposes, we will only be interested in <b>and</b> and <b>xor</b> .

<b>and</b> takes in two inputs and produces one output, given by the following logic table: 

a|b|a&b
:---:|:--:|:-----:
 0 | 0 |  0  
 1 | 0 |  0  
 0 | 1 |  0  
 1 | 1 |  1  
Julia's `&` is the bitwise operation and.  That means if I combine two numbers, it states the overlap between the two. 1 overlaps with 1; 2 overlaps with 2; 3 overlaps with 2 and 1.

We will use this to compute magnetization.

In [9]:
println(bin(1,4),"\t", bin(3,4),"\t", bin(1&3,4))

0001	0011	0001


In [24]:
m=zeros(psi)

println("String \tp&powers2 \tNormed \t\tMagentization")
for i in 1:length(psi)
    m[i]=sum(round(Int,(psi[i]&powers2)./powers2))
    
    println(bin(psi[i],n+1),"\t",psi[i]&powers2,"\t"
    ,round(Int,(psi[i]&powers2)./powers2),"\t",m[i]) 
end

String 	p&powers2 	Normed 		Magentization
0000	[0,0,0,0]	[0,0,0,0]	0
0001	[1,0,0,0]	[1,0,0,0]	1
0010	[0,2,0,0]	[0,1,0,0]	1
0011	[1,2,0,0]	[1,1,0,0]	2
0100	[0,0,4,0]	[0,0,1,0]	1
0101	[1,0,4,0]	[1,0,1,0]	2
0110	[0,2,4,0]	[0,1,1,0]	2
0111	[1,2,4,0]	[1,1,1,0]	3


## Masks and Permuting Indices

The off diagonal elements of the Hamiltonian are the permutation of two neighboring elements in the array.  We can permute two indices by combining a mask with a bitwise XOR `$` .

In [18]:
mask=3
testp=1
println("Mask \ttest \tmasked test")
println(bin(mask,2),'\t',bin(testp,2),'\t',bin(testp$mask,2))

Mask 	test 	masked test
11	01	10


But we bound always be just switching the first two digits, so we need a set of masks that can flip any adjacent spins.  I create this by summing together padded powers of two in order to get the 11 in the correct location.

In [36]:
mask=[0;powers2]+[powers2;0]
mask=mask[2:end-1]

println("Mask base10 \tMask Binary \tSummed from")
for i in 1:length(mask)
    println(mask[i],"\t\t",bin(mask[i],n+1),"\t\t",bin(powers2[i],n+1),"\t",bin(powers2[i+1],n+1))
end

Mask base10 	Mask Binary 	Summed from
3		0011		0001	0010
6		0110		0010	0100
12		1100		0100	1000


So now lets test how the first of our three masks behaves:
We know that if the mask changes a 01 for a 10 (or vice versa) that the overall magnetization will not be changed.  So, we test is our mask is successful by comparing the remaining magnetization.  The rows offset by two spaces have matching magnetizations.

In [56]:
println("Psi \tPsi \tMasked \t Masked\t \tmPsi  \tmMasked")
for p in psi
    if m[p+1]==m[p$mask[1]+1]
        println("  ",p,"\t  ",bin(p,4),"\t  ",p $ mask[1],"\t  ",bin(p$mask[1],4),"\t\t  ",m[p+1],"\t  ",m[p$mask[1]+1]) 
    else
        println(p,'\t',bin(p,4),'\t',p $ mask[1],'\t',bin(p$mask[1],4),"\t\t",m[p+1],"\t",m[p$mask[1]+1]) 
    end
end

Psi 	Psi 	Masked 	 Masked	 	mPsi  	mMasked
0	0000	3	0011		0	2
  1	  0001	  2	  0010		  1	  1
  2	  0010	  1	  0001		  1	  1
3	0011	0	0000		2	0
4	0100	7	0111		1	3
  5	  0101	  6	  0110		  2	  2
  6	  0110	  5	  0101		  2	  2
7	0111	4	0100		3	1


In [57]:
println("Psi \tPsi \tMasked \t Masked\t \tmPsi  \tmMasked")
for p in psi
    if m[p+1]==m[p$mask[2]+1]
        println("  ",p,"\t  ",bin(p,4),"\t  ",p $ mask[2],"\t  ",bin(p$mask[2],4),"\t\t  ",m[p+1],"\t  ",m[p$mask[2]+1]) 
    else
        println(p,'\t',bin(p,4),'\t',p $ mask[2],'\t',bin(p$mask[2],4),"\t\t",m[p+1],"\t",m[p$mask[2]+1]) 
    end
end

Psi 	Psi 	Masked 	 Masked	 	mPsi  	mMasked
0	0000	6	0110		0	2
1	0001	7	0111		1	3
  2	  0010	  4	  0100		  1	  1
  3	  0011	  5	  0101		  2	  2
  4	  0100	  2	  0010		  1	  1
  5	  0101	  3	  0011		  2	  2
6	0110	0	0000		2	0
7	0111	1	0001		3	1


In [ ]:
println("Psi \tPsi \tMasked \t Masked\t \tmPsi  \tmMasked")

Obviously in situations like `0`, that's not a permutation going from `0000` to `0011`.  We need to find a set of two neighbors that are different from each other first, before we then mask them to find the permuted value.

In [37]:
mask[2]

6